In [40]:
### Library Import Initialization
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.integrate import quad
import warnings
warnings.filterwarnings("ignore")


In [41]:
### Load in Stock Data
def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
start_data = '2022-01-01'
stock_data = import_stock_data(tickers, start_data)
# Get most recent stock price
S_0 = stock_data.iloc[1, -1]
print('The current stock price is:', round(S_0, 4))

# Set the Strike Price (K) based on the current asset price (S_0)
K = 142
print('The Strike Price is set to:', K)


[*********************100%%**********************]  1 of 1 completed

The current stock price is: 144.2523
The Strike Price is set to: 142


In [42]:
### Calculate Log Returns and Historical Volatility (sigma)
def get_log_returns(data):
    return np.log(data / data.shift(1))

def volatility(log_returns):
    std_dev = log_returns.std()
    sigma = np.sqrt(252) * std_dev  # Annualized volatility (252 trading days per year)
    return sigma

# Assuming stock_data is already defined
log_returns = get_log_returns(stock_data)
sigma = volatility(log_returns)
v_0 = sigma ** 2
print('Sigma: ', sigma)
print('v_0', v_0)

Sigma:  GOOG    0.338159
dtype: float64
v_0 GOOG    0.114351
dtype: float64


In [43]:
### Compute the Characteristic Function
# See whitepaper for characteristic equation
def char_func(u, S_0, r, T, k, theta, sigma, rho, v_0):
    i = 1j
    b = k - rho * sigma * i * u
    d = np.sqrt(b ** 2 + (sigma ** 2) * (i * u + u ** 2))
    g = (b - d) / (b + d)
    e_dt = np.exp(-d * T)
    C = r * i * u * T + (k * theta / (sigma ** 2)) * ((b - d) * T - 2 * np.log((1 - g * e_dt) / (1 - g)))
    D = ((b - d) / (sigma ** 2)) * ((1 - e_dt) / (1 - g * e_dt))
    
    char_eqn = np.exp(C + D * v_0 + i * u * np.log(S_0))

    return char_eqn


In [44]:
### Compute Probability P_1
# See whitepaper for integral equation to estimate probability P_1
def p_1(S0, K, T, r, kappa, theta, sigma, rho, v0):
    def integrand(u):
        numerator = np.exp(-1j * u * np.log(K)) * char_func(u - 1j, S0, r, T, kappa, theta, sigma, rho, v0)
        denominator = 1j * u * char_func(-1j, S0, r, T, kappa, theta, sigma, rho, v0)
        return np.real(numerator / denominator).squeeze()

    result, error = quad(integrand, 0, np.inf)
    return 0.5 + (1 / np.pi) * result

### Compute Probability P_2
# See whitepaper for integral equation to estimate probability P_2
def p_2(S0, K, T, r, kappa, theta, sigma, rho, v0):
    def integrand(u):
        numerator = np.exp(-1j * u * np.log(K)) * char_func(u, S0, r, T, kappa, theta, sigma, rho, v0)
        denominator = 1j * u
        return np.real(numerator / denominator).squeeze()

    result, error = quad(integrand, 0, np.inf)
    return 0.5 + (1 / np.pi) * result


In [45]:
### Compute the Heston Approximation for a Call and Put Option
def heston_approx(S_0, K, T, r, k, theta, sigma, rho, v_0, option_type):
    P_1 = p_1(S_0, K, T, r, k, theta, sigma, rho, v_0)
    P_2 = p_2(S_0, K, T, r, k, theta, sigma, rho, v_0)
    
    if option_type == 'call':
        call_price = S_0 * P_1 - K * np.exp(-r * T) * P_2
        return call_price
    elif option_type == 'put':
        put_price = S_0 * (P_1 - 1) + K * np.exp(-r * T) * (1 - P_2)
        return put_price
    else:
        raise ValueError("Invalid option type. Choose 'Call' or 'Put'.")


In [46]:
### Initialize Parameters
# Kappa = Mean-reversion speed in the Heston model
k = 2 
# Long-term average variance (squared volatility) in the Heston model
theta = v_0 ** 2  
print('Theta was calculated as: ', theta)
# rho - correlation parameter
rho = -0.5
# Time to Expiration (years)
T = 1
# Interest rate
r = 0.05
# Option type (Call or Put)
option_type = 'put'

### Function Call
option_price = heston_approx(S_0, K, T, r, k, theta, sigma, rho, v_0, option_type)
print(option_type + " Option Price Estimate:", round(option_price, 4))


Theta was calculated as:  GOOG    0.013076
dtype: float64
put Option Price Estimate: 9.0187


In [47]:
### Test Case
S_0 = 100
K_call = 105
K_put = 95
T = 1
r = 0.05
k = 1.5
theta = 0.03
sigma = 0.2
rho = -0.7
v_0 = 0.04

call_price = heston_approx(S_0, K_call, T, r, k, theta, sigma, rho, v_0, 'call')
put_price = heston_approx(S_0, K_put, T, r, k, theta, sigma, rho, v_0, 'put')
print('Call Option Price:', round(call_price, 4))
print('Put Option Price:', round(put_price, 4))


Call Option Price: 7.301
Put Option Price: 3.5164
